# Generate UMAP coordinates for each plate

## Import libraries

In [1]:
import glob
import pathlib
import pandas as pd
import umap

from pycytominer import feature_select
from pycytominer.cyto_utils import infer_cp_features


## Set constants

In [2]:
# Set constants
umap_random_seed = 0
umap_n_components = 2

output_dir = pathlib.Path("results")
output_dir.mkdir(parents=True, exist_ok=True)


## Create list of paths to feature selected data per plate

In [3]:
# Set input paths
data_dir = pathlib.Path("../../../3.processing_features/data/single_cell_profiles/")

# Select only the feature selected files
file_suffix = "*sc_feature_selected.parquet"

# Obtain file paths for all feature selected plates
fs_files = glob.glob(f"{data_dir}/{file_suffix}")
fs_files


['../../../3.processing_features/data/single_cell_profiles/Plate_1_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_3_prime_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_4_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_5_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_2_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_3_sc_feature_selected.parquet']

In [4]:
# Load feature data into a dictionary, keyed on plate name
cp_dfs = {x.split("/")[-1]: pd.read_parquet(x) for x in fs_files}

# Print out useful information about each dataset
print(cp_dfs.keys())
[cp_dfs[x].shape for x in cp_dfs]


dict_keys(['Plate_1_sc_feature_selected.parquet', 'Plate_3_prime_sc_feature_selected.parquet', 'Plate_4_sc_feature_selected.parquet', 'Plate_5_sc_feature_selected.parquet', 'Plate_2_sc_feature_selected.parquet', 'Plate_3_sc_feature_selected.parquet'])


[(241, 849),
 (5506, 1146),
 (7308, 1163),
 (5793, 1174),
 (1714, 856),
 (11286, 1171)]

## Generate UMAP coordinates for each plate

**Note:** Only metadata that is common between plates are included in final data frame.

In [5]:
desired_columns = ["Metadata_Plate","Metadata_Well", "Metadata_Site", "Metadata_number_of_singlecells", "Metadata_genotype"]

# Fit UMAP features per dataset and save
for plate in cp_dfs:
    plate_name = pathlib.Path(plate).stem
    print("UMAP embeddings being generated for", plate_name)

     # Make sure to reinitialize UMAP instance per plate
    umap_fit = umap.UMAP(
        random_state=umap_random_seed,
        n_components=umap_n_components
    )

    # Make sure NA columns have been removed
    cp_df = cp_dfs[plate]
    cp_df = feature_select(
        cp_df,
        operation="drop_na_columns",
        na_cutoff=0
    )

    # Make sure that the Plate_3_prime has correct name in Metadata_Plate column
    if plate_name.replace("_sc_feature_selected", "") == "Plate_3_prime":
        cp_df["Metadata_Plate"] = "Plate_3_prime"

    # Remove rows with genotype HET for Plate_5
    if plate_name.replace("_sc_feature_selected", "") == "Plate_5":
        cp_df = cp_df[cp_df["Metadata_genotype"] != "HET"]
    
    # Process cp_df to separate features and metadata
    cp_features = infer_cp_features(cp_df)
    meta_features = infer_cp_features(cp_df, metadata=True)
    filtered_meta_features = [feature for feature in meta_features if feature in desired_columns]
    
    # Fit UMAP and convert to pandas DataFrame
    embeddings = pd.DataFrame(
        umap_fit.fit_transform(cp_df.loc[:, cp_features]),
        columns=[f"UMAP{x}" for x in range(0, umap_n_components)]
    )
    print(embeddings.shape)
    
    # Combine with metadata
    cp_umap_with_metadata_df = pd.concat([
        cp_df.loc[:, filtered_meta_features].reset_index(drop=True),
        embeddings
    ], axis=1)
    
    # randomize the rows of the dataframe to plot the order of the data evenly
    cp_umap_with_metadata_df = cp_umap_with_metadata_df.sample(frac=1, random_state=0)

    # Generate output file and save
    output_umap_file = pathlib.Path(output_dir, f"UMAP_{plate_name}.tsv")
    cp_umap_with_metadata_df.to_csv(output_umap_file, index=False, sep="\t")


UMAP embeddings being generated for Plate_1_sc_feature_selected


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(241, 2)
UMAP embeddings being generated for Plate_3_prime_sc_feature_selected


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(5506, 2)
UMAP embeddings being generated for Plate_4_sc_feature_selected


...

(5793, 2)
UMAP embeddings being generated for Plate_2_sc_feature_selected


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(1714, 2)
UMAP embeddings being generated for Plate_3_sc_feature_selected


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(11286, 2)


In [6]:
# Print an example output file
print(cp_umap_with_metadata_df.shape)
cp_umap_with_metadata_df.head()


(11286, 7)


,Metadata_Well,Metadata_Site,Metadata_number_of_singlecells,Metadata_genotype,Metadata_Plate,UMAP0,UMAP1
4136,D4,8,636,WT,Plate_3,8.818437,4.097239
5457,B10,24,86,Null,Plate_3,9.012610,6.500041
10346,F12,17,647,Null,Plate_3,8.104309,5.912871
3701,D4,1,636,WT,Plate_3,10.596231,3.488836
9596,E12,14,607,Null,Plate_3,9.566523,3.041514


In [7]:
# Set input paths
data_dir = pathlib.Path("../../../3.processing_features/data/single_cell_profiles/")

# Select only the feature selected files
fs_suffix = "*sc_feature_selected.parquet"

# Obtain file paths for all feature selected plates
fs_files = glob.glob(f"{data_dir}/{fs_suffix}")
fs_files

['../../../3.processing_features/data/single_cell_profiles/Plate_1_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_3_prime_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_4_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_5_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_2_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_3_sc_feature_selected.parquet']

In [8]:
# Select file paths for plates 5, 3, and 3 prime only
selected_plates = ["Plate_5", "Plate_3", "Plate_3_prime"]

# Filter and concatenate the selected plates
selected_dfs = []
for file_path in fs_files:
    plate_name = pathlib.Path(file_path).stem.replace("_sc_feature_selected", "")

    # Only read in selected plates
    if plate_name in selected_plates:
        df = pd.read_parquet(file_path)

        selected_dfs.append(df)

In [9]:
# Get the column names of all DataFrames in selected_dfs
column_sets = [set(df.columns) for df in selected_dfs]

# Find the common column names across all DataFrames
common_columns = list(set.intersection(*column_sets))

# Exclude columns that start with "Metadata" to print the number of features
feature_columns = [col for col in common_columns if not col.startswith("Metadata")]

# Print length of only features
len(feature_columns)

907

In [10]:
# Filter each DataFrame in selected_dfs to include only common columns
selected_dfs_filtered = [df.loc[:, common_columns] for df in selected_dfs]

# Concatenate the filtered dataframes along the rows
concatenated_df = pd.concat(selected_dfs_filtered, ignore_index=True)

# Save the concatenated dataframe to a file
output_concatenated_file = pathlib.Path(output_dir, "concatenated_norm_fs_plates_5_3_3prime.parquet")
concatenated_df.to_parquet(output_concatenated_file, index=False)

print(concatenated_df.shape)
concatenated_df.head()


(22585, 924)


,Cells_RadialDistribution_ZernikePhase_GFP_6_2,Cells_RadialDistribution_ZernikeMagnitude_DAPI_3_1,Cells_RadialDistribution_ZernikePhase_DAPI_9_7,Cytoplasm_Intensity_IntegratedIntensity_DAPI,Nuclei_RadialDistribution_ZernikePhase_CY5_8_8,Cytoplasm_RadialDistribution_ZernikePhase_GFP_5_3,Nuclei_RadialDistribution_ZernikeMagnitude_GFP_7_7,Nuclei_RadialDistribution_ZernikePhase_DAPI_7_7,Nuclei_RadialDistribution_ZernikeMagnitude_RFP_8_0,Cytoplasm_RadialDistribution_ZernikePhase_RFP_2_2,...,Cells_RadialDistribution_RadialCV_GFP_2of4,Cells_RadialDistribution_ZernikePhase_GFP_7_3,Cells_RadialDistribution_ZernikeMagnitude_DAPI_6_0,Cytoplasm_RadialDistribution_ZernikeMagnitude_GFP_9_5,Cells_RadialDistribution_ZernikePhase_RFP_9_3,Nuclei_RadialDistribution_ZernikePhase_RFP_8_2,Cytoplasm_RadialDistribution_ZernikePhase_RFP_6_4,Cytoplasm_RadialDistribution_ZernikeMagnitude_GFP_7_1,Nuclei_RadialDistribution_ZernikePhase_GFP_7_3,Nuclei_RadialDistribution_ZernikePhase_GFP_5_3
0,-1.677435,0.625136,1.518357,0.216370,-0.654427,0.402476,0.837912,1.090303,1.180208,1.276914,...,1.749038,-1.417871,-1.241596,0.315861,0.400919,1.196219,0.059943,1.685460,-0.529717,-1.603872
1,0.958254,0.313378,-0.861874,0.160026,-1.395179,-1.007452,-0.502066,0.104108,1.844138,0.257564,...,-0.644971,0.961738,-0.065138,0.298905,-0.432340,-0.422283,1.566175,-0.962706,-0.044767,0.417456
2,0.248763,-0.850626,-0.905618,0.339683,-1.343707,0.304697,-0.554036,-1.306485,-0.513942,0.480819,...,-0.532064,-1.466929,0.131230,-1.023434,1.527015,1.472545,-1.136578,0.840905,1.334838,-1.339474
3,-1.384300,0.749956,1.415600,0.347419,0.028354,0.931277,-0.192492,-1.031757,0.053700,0.991371,...,2.280827,-0.833413,-0.096487,-0.500810,1.412816,-0.033132,-0.784797,0.275493,-0.932890,0.651154
4,-1.689671,-0.185670,-0.337484,0.390760,-1.602552,-1.314188,0.809491,1.238281,0.805860,1.694736,...,-0.671582,0.428789,-1.307493,-0.529744,-0.951060,-1.595130,0.693836,0.225127,0.137821,-0.322376


In [11]:
desired_columns = ["Metadata_Plate","Metadata_Well", "Metadata_Site", "Metadata_number_of_singlecells", "Metadata_genotype"]

# Make sure to reinitialize UMAP instance
umap_fit = umap.UMAP(
    random_state=umap_random_seed,
    n_components=umap_n_components
)

# Process cp_df to separate features and metadata
cp_features = infer_cp_features(concatenated_df)
meta_features = infer_cp_features(concatenated_df, metadata=True)
filtered_meta_features = [feature for feature in meta_features if feature in desired_columns]

# Fit UMAP and convert to pandas DataFrame
embeddings = pd.DataFrame(
    umap_fit.fit_transform(concatenated_df.loc[:, cp_features]),
    columns=[f"UMAP{x}" for x in range(0, umap_n_components)]
)
print(embeddings.shape)

# Combine with metadata
cp_umap_with_metadata_df = pd.concat([
    concatenated_df.loc[:, filtered_meta_features],
    embeddings
], axis=1)

# randomize the rows of the dataframe to plot the order of the data evenly
cp_umap_with_metadata_df = cp_umap_with_metadata_df.sample(frac=1, random_state=0)

# Generate output file and save
output_umap_file = pathlib.Path(output_dir, f"UMAP_concat_model_plates_sc_feature_selected.tsv")
cp_umap_with_metadata_df.to_csv(output_umap_file, index=False, sep="\t")


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(22585, 2)
